# Impact Quantization

In [1]:
# imports
import os
import numpy as np
import pickle

In [2]:
# load impact points
load_dir = os.path.join('data', 'impact_points')
vid_name = 'test'
impact_points = np.load(os.path.join(load_dir, f'{vid_name}_impact_points.npy'))
time_stamps = np.load(os.path.join(load_dir, f'{vid_name}_time_stamps.npy'))

In [3]:
# function to quantize impact points
def quantize_impact_points(impact_points, time_stamps):
    assert len(impact_points) == len(time_stamps)
    n_frames, n_limbs = impact_points.shape
    
    quantized_impacts = []
    for l in range(n_limbs):
        quantized_impacts.append(np.arange(4 * l, 4 * (l + 1)))
    
    return quantized_impacts

# testing
quantized_impacts = quantize_impact_points(impact_points, time_stamps)
print(quantized_impacts)

[array([0, 1, 2, 3]), array([4, 5, 6, 7]), array([ 8,  9, 10, 11]), array([12, 13, 14, 15])]


In [4]:
# save results to file
save_dir = os.path.join('data', 'quantized_impacts')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
vid_name = 'test'

with open(os.path.join(save_dir, f'{vid_name}_quantized_impacts.pkl'), 'wb') as f:
    pickle.dump(quantized_impacts, f)